In [32]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
normal = pd.read_csv("C:/Users/tlseh/Desktop/WADI_14days_new.csv")

In [3]:
normal_drop = normal.drop(['Row','Date','Time'],axis='columns')

In [4]:
normal_drop_1 = normal_drop[0:1000] # 앞에 1000개만 우선 해보기 위해서 부분 선택
normal_drop_1 = normal_drop_1.dropna(axis=1) # 값이 없는 열 제거 여기서는 행 4개가 제거되었다.

In [5]:
normal_drop_1

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,1_MV_001_STATUS,...,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
1,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
2,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,47.8911,1,...,1,1,1,1,1,1,1,67.9651,1,0.68
3,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
4,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,47.7503,1,...,1,1,1,1,1,1,1,67.1948,1,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
996,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
997,174.938,0.559481,11.7524,484.411,0.286654,1.907730,0,0,40.5656,2,...,1,1,1,1,1,1,1,63.8467,1,0.68
998,174.213,0.541483,11.7854,482.401,0.286422,2.003600,0,0,40.8926,2,...,1,1,1,1,1,1,1,63.9204,1,0.68


In [6]:
normal_drop_1.isnull().sum().sum() # 성분에서 NaN이 있으면 1이상의 값이 나오게 된다. pd.DataFrame.isnull().sum().sum()

0

In [7]:
from sklearn import preprocessing       # 정규화를 반드시 진행해야 하는 건지?
min_max_scaler = preprocessing.MinMaxScaler()

x = normal_drop_1.values
x_scaled = min_max_scaler.fit_transform(x)
normal_drop_1_scaled = pd.DataFrame(x_scaled)

print(normal_drop_1_scaled)

normal_drop_1 = normal_drop_1_scaled   # 정규화 된 데이터를 넣어주어야함! sklearn 사용하면 정규화 복구하는데도 편리하다.

normal_drop_1 = np.array(normal_drop_1) # Dataframe을 numpy로 변환

          0         1         2         3         4         5    6    7    \
0    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
1    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
2    0.006593  0.529401  0.252453  0.919934  0.824437  0.000094  0.0  0.0   
3    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
4    0.006593  0.470590  0.240948  0.921092  0.827501  0.000118  0.0  0.0   
..        ...       ...       ...       ...       ...       ...  ...  ...   
995  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
996  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
997  0.662908  0.235287  0.849746  0.083127  0.009064  0.923259  0.0  0.0   
998  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   
999  0.537127  0.147052  0.961421  0.000000  0.003090  0.969679  0.0  0.0   

          8    9    ...  113  114  115  116  117  118  119       120  121  

In [8]:
#  내가 입력차원을 잘못 생각함, 반대로 해주어야한다.

# normal_drop_1.shape
# print("before transpose :{}".format(normal_drop_1.shape)) # 데이터셋 입력전 rnn 모양을 맞춰주기 위해서 행열 transpose
# normal_drop_1 = np.transpose(normal_drop_1)
# print("after transpose :{}".format(normal_drop_1.shape))

In [9]:
normal_drop_1 = torch.from_numpy(normal_drop_1).unsqueeze(0) # 배치 차원을 추가 해주기 위해서 첫번째 위치에 차원 추가

In [10]:
normal_drop_1.shape

torch.Size([1, 1000, 123])

In [11]:
def Window(data,window,step):   # window 구현을 어떻게 해야 할지 몰라서 함수로 구현
    batch_size = (data.shape[1] - window) // step # 두번째 인덱스가 전체 샘플 개수
    Window_complete = torch.tensor([])

    for index in range(batch_size):
        x = torch.clone(data[:, step * index : window + step * index,:])
        Window_complete = torch.cat((Window_complete,x),dim=0) 

    return Window_complete # (batch,window,sensor)

In [12]:
window=5
step=3
data = Window(normal_drop_1,window=window,step=step)
print(data.shape)

torch.Size([331, 5, 123])


In [13]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [14]:
class CustomDataset(Dataset):   # 데이터셋에 굳이 넣어야 하는건지?
  def __init__(self,data):
      self.data = data

  def __getitem__(self, idx): 
    return self.data[idx]

  def __len__(self): 
    return len(self.data)


In [15]:
dataset = CustomDataset(data)

In [16]:
dataset.data.shape

torch.Size([331, 5, 123])

In [17]:
SEQ_LEN = dataset.data.shape[1]
INPUT_SIZE = dataset.data.shape[2]
BATCH_SIZE = 3

In [18]:
dataloader = DataLoader(dataset,batch_size=BATCH_SIZE, shuffle=False,drop_last=True)

In [19]:
from torch import nn

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim=32):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = embedding_dim * 2
        self.embedding_dim = embedding_dim 

        
        self.rnn1 = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_dim,num_layers=1,batch_first=True)
        self.rnn2 = nn.LSTM(input_size=self.hidden_dim,hidden_size=self.embedding_dim,num_layers=1,batch_first=True)

    def forward(self, encoder_output):           # (batch size, window, sensor=123)
        encoder_output = encoder_output.float()
        encoder_output,(_,_) = self.rnn1(encoder_output)
        encoder_output, (hidden,_) = self.rnn2(encoder_output) # encoder_output = (batch size, window, embedding_dim)

        return encoder_output, hidden    # hidden = (num_layers=1, batch size, embedding_dim)

In [50]:
class Attention(nn.Module):
    def __init__(self,enc_embedding_dim, dec_embedding_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(enc_embedding_dim *2,dec_embedding_dim)
        self.v = nn.Linear(dec_embedding_dim, 1, bias = False)
        

    def forward(self,encoder_output, hidden):   # encoder_output = (batch size, window, embedding_dim)
                                                # hidden = (num_layers=1, batch size, embedding_dim)

        hidden = hidden.premute(1,0,2).repeat(1,window,1)  # hidden = (batch size, window, embedding_dim)
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_output),dim = 2)))
        # enegry = (batch size, window, dec_embedding_dim)

        attention = self.v(energy) # attention = (batch size, window, 1)
        attention = F.softmax(attention, dim=1)
        
        attention = torch.mul(attention,encoder_output)
        return  attention   # attention = (batch size, window, embedding_dim)

In [59]:
class Decoder(nn.Module):
    def __init__(self,input_size, output_size, attention):
        super(Decoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = input_size * 2
        self.output_size = output_size

        self.attention = attention
        self.rnn1 = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_dim,batch_first=True)
        self.rnn2 = nn.LSTM(input_size=self.hidden_dim,hidden_size=self.output_size,batch_first=True)

    def forward(self,input, hidden, encoder_outputs):# hidden = (num_layers=1, batch size, embedding_dim)
        hidden = hidden.permute(1,0,2)    # hidden = (batch size, num_layers=1, embedding_dim)
        hidden = hidden.repeat(1,window,1)# hidden = (batch size, window, embedding_dim)
        attention = self.attention(encoder_outputs, hidden)
        x = hidden + attention
        x,(_,_) = self.rnn1(x)
        x,(_,_) = self.rnn2(x)  # (batch_size,window,output_size)
        return x 

In [23]:
class Autoencoder(nn.Module):

  def __init__(self,INPUT_SIZE, EMBEDDING_DIM=32):
    super(Autoencoder, self).__init__()

    self.encoder = Encoder(input_size=INPUT_SIZE, embedding_dim=EMBEDDING_DIM)
    self.decoder = Decoder(input_size=EMBEDDING_DIM, output_size=INPUT_SIZE)

  def forward(self, x):
    encoder_output, hidden = self.encoder(x)
    x = self.decoder(x)

    return x

In [24]:
model = Autoencoder(INPUT_SIZE,32)
optimizer = torch.optim.Adam(model.parameters())
#criterion = nn.CrossEntropyLoss()
criterion = nn.L1Loss(reduction='sum')

In [27]:
#for i in range(100):
for i, batch in enumerate(dataloader):
        
    optimizer.zero_grad()
    
    output = model(batch)
    
    loss = criterion(output, batch)
    
    loss.backward()

    if i % 10 == 0:
        print("NOW index : {}, LOSS : {}".format(i, loss))

    optimizer.step()
    

NOW index : 0, LOSS : 369.7671813964844
NOW index : 10, LOSS : 364.8744812011719
NOW index : 20, LOSS : 411.13262939453125
NOW index : 30, LOSS : 258.9564208984375
NOW index : 40, LOSS : 163.02294921875
NOW index : 50, LOSS : 100.57623291015625
NOW index : 60, LOSS : 82.93132019042969
NOW index : 70, LOSS : 418.251708984375
NOW index : 80, LOSS : 216.02322387695312
NOW index : 90, LOSS : 114.42770385742188
NOW index : 100, LOSS : 95.28270721435547
